In [1]:
import pygame
import math

pygame.init()

WIDTH, HEIGHT = 1200, 800
TOOLBAR_HEIGHT = 50
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Paint')
screen.fill((255, 255, 255)) 
base_layer = pygame.Surface((WIDTH, HEIGHT))
base_layer.fill((255, 255, 255))  
clock = pygame.time.Clock()

colorRED = (255, 0, 0)
colorGREEN = (0, 255, 0)
colorBLUE = (0, 0, 255)
colorPURPLE = (255,0,255)
colorYELLOW = (255, 255, 0)
colorWHITE = (255, 255, 255)
colorBLACK = (0, 0, 0)
colors = [colorBLACK, colorRED, colorBLUE, colorGREEN, colorPURPLE, colorYELLOW]
curr_color = colors[0]


thicknesses = [15, 5, 5, 5, 5, 5, 5, 5]  
mod = 1  
LMBpressed = False


mouse_x, mouse_y = pygame.mouse.get_pos()
curr_x, curr_y = mouse_x, mouse_y
prev_x, prev_y = mouse_x, mouse_y


def calculate_rect(x1, y1, x2, y2):
    return pygame.Rect(min(x1, x2), min(y1, y2), abs(x1 - x2), abs(y1 - y2))

def calculate_right_triangle_points(x1, y1, x2, y2): # Right Triangle
        return [(x1, y1), (x2, y2), (x1, y2)]

def calculate_equilateral_triangle_points(x1, y1, x2, y2): # Equilateral triangle
        base_mid_x = (x1 + x2) / 2
        base_mid_y = (y1 + y2) / 2
        side_length = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** 0.5
        height = (math.sqrt(3) / 2) * side_length
        if y2 > y1:  
            top_y = y1 - height
        else:
            top_y = y1 + height
        return [(x1, y2), (x2, y2), (base_mid_x, top_y)]

def calculate_rhombus_points(x1, y1, x2, y2):
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    width = abs(x2 - x1) / 2
    height = abs(y2 - y1) / 2
    return [
        (center_x, y1),     
        (x2, center_y),      
        (center_x, y2),      
        (x1, center_y)       
    ]

def draw_figure(figure_index, surface):
    if figure_index == 0:  # Eraser
        pygame.draw.circle(surface, colorWHITE, (curr_x, curr_y), thicknesses[0] // 2)
        pygame.draw.line(surface, colorWHITE, (prev_x, prev_y), (curr_x, curr_y), thicknesses[0])
    
    elif figure_index == 1:  # Brush
        pygame.draw.circle(surface, curr_color, (curr_x, curr_y), thicknesses[1] // 2)
        pygame.draw.line(surface, curr_color, (prev_x, prev_y), (curr_x, curr_y), thicknesses[1])
    
    elif figure_index == 2:  # Line
        pygame.draw.line(surface, curr_color, (prev_x, prev_y), (curr_x, curr_y), thicknesses[2])
    
    elif figure_index == 3:  # Rectangle
        pygame.draw.rect(surface, curr_color, calculate_rect(prev_x, prev_y, curr_x, curr_y), thicknesses[3])
    
    elif figure_index == 4:  # Circle
        radius = int(((curr_x - prev_x)**2 + (curr_y - prev_y)**2)**0.5)
        pygame.draw.circle(surface, curr_color, (prev_x, prev_y), radius, thicknesses[4])

    elif figure_index == 5: # Right Triangle
        points = calculate_right_triangle_points(prev_x, prev_y, curr_x, curr_y)
        pygame.draw.polygon(surface, curr_color, points, thicknesses[5])
        pygame.draw.circle(surface, curr_color, points[0], thicknesses[5] // 2)
        pygame.draw.circle(surface, curr_color, points[1], thicknesses[5] // 2)
        pygame.draw.circle(surface, curr_color, points[2], thicknesses[5] // 2)

    elif figure_index == 6: # Equilateral triangle
        points = calculate_equilateral_triangle_points(prev_x, prev_y, curr_x, curr_y)
        pygame.draw.polygon(surface, curr_color, points, thicknesses[6])
        pygame.draw.circle(surface, curr_color, points[0], thicknesses[6] // 2)
        pygame.draw.circle(surface, curr_color, points[1], thicknesses[6] // 2)
        pygame.draw.circle(surface, curr_color, points[2], thicknesses[6] // 2)
    
    elif figure_index == 7: # Rhoumbus
        points = calculate_rhombus_points(prev_x, prev_y, curr_x, curr_y)
        pygame.draw.polygon(surface, curr_color, points, thicknesses[7])
        pygame.draw.circle(surface, curr_color, points[1], thicknesses[7] // 2)
        pygame.draw.circle(surface, curr_color, points[0], thicknesses[7] // 2)
        pygame.draw.circle(surface, curr_color, points[2], thicknesses[7] // 2)
        pygame.draw.circle(surface, curr_color, points[3], thicknesses[7] // 2)

def clear_screen():
    base_layer.fill(colorWHITE)
    screen.fill(colorWHITE)

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        if event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
            LMBpressed = True
            curr_x, curr_y = event.pos
            prev_x, prev_y = event.pos

        if event.type == pygame.MOUSEMOTION and LMBpressed:
            if mod in [0, 1]:  # Eraser and Brush draw immediately
                prev_x, prev_y = curr_x, curr_y
                curr_x, curr_y = event.pos
                draw_figure(mod, base_layer)
            elif mod in [2, 3, 4, 5, 6, 7]:  # Line, Rectangle, Circle etc. are temporarily drawn on the screen
                curr_x, curr_y = event.pos

        if event.type == pygame.MOUSEBUTTONUP and event.button == 1:
            LMBpressed = False
            draw_figure(mod, base_layer)   # Fix the drawing on the canvas

        if event.type == pygame.KEYDOWN: 
            if event.key == pygame.K_1: mod = 0  # Eraser
            if event.key == pygame.K_2: mod = 1  # Brush
            if event.key == pygame.K_3: mod = 2  # Line
            if event.key == pygame.K_4: mod = 3  # Rectangle
            if event.key == pygame.K_5: mod = 4  # Circle
            if event.key == pygame.K_6: mod = 5  # Right triangle 
            if event.key == pygame.K_7: mod = 6  # Equilateral triangle
            if event.key == pygame.K_8: mod = 7  # Rhoumbus
            
            if event.key == pygame.K_d: curr_color = colors[0]
            if event.key == pygame.K_r: curr_color = colors[1]
            if event.key == pygame.K_b: curr_color = colors[2]
            if event.key == pygame.K_g: curr_color = colors[3]
            if event.key == pygame.K_p: curr_color = colors[4]
            if event.key == pygame.K_y: curr_color = colors[5]
            if event.key == pygame.K_c:  # Clear screen with 'c' key
                clear_screen()

        keys = pygame.key.get_pressed()
        if keys[pygame.K_KP_PLUS ] or keys[pygame.K_RIGHTBRACKET]:
            if thicknesses[mod] <= 64:
                thicknesses[mod] += 1
        if keys[pygame.K_KP_MINUS]or keys[pygame.K_LEFTBRACKET]:
            if thicknesses[mod] >= 2:
                thicknesses[mod] -= 1

    screen.blit(base_layer, (0, 0))
    
    if LMBpressed and mod in [2, 3, 4, 5, 6, 7]:
        draw_figure(mod, screen)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


2025-04-02 12:26:39.103 python[5123:12559241] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-02 12:26:39.103 python[5123:12559241] +[IMKInputSession subclass]: chose IMKInputSession_Modern
